네이버영화평점
==
- 감성분석
- 네이버 영화평점 데이터(https://github.com/e9t/nsmc)

In [65]:
!nvidia-smi

Sun Oct 28 23:13:50 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:03:00.0  On |                  N/A |
| 24%   43C    P8    17W / 250W |     18MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   35C    P8    17W / 250W |      2MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

# 로지스틱회귀를 이용한 감성분석

In [16]:


import konlpy
import pandas as pd
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import pickle
import os.path

df_train = pd.read_csv('data/ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('data/ratings_test.txt', delimiter='\t', keep_default_na=False)

df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
text_train, y_train = df_train['document'].as_matrix(), df_train['label'].as_matrix()
text_test = df_test['document'].as_matrix()
y_test = df_test['label'].as_matrix()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [17]:
twitter_tag = Twitter()

cv = TfidfVectorizer(tokenizer=twitter_tokenizer)

In [20]:
lr = LogisticRegression()

x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)
result = lr.fit(x_train,y_train)

In [22]:
feature_names = cv.get_feature_names()
print("특성:\n{}".format(feature_names[8000:8020]))

특성:
['고달픔', '고담', '고대', '고대로', '고대소설', '고도', '고도리', '고독', '고독감', '고독고독', '고동', '고두림', '고두심', '고드헌터', '고든', '고등', '고등법원', '고등어', '고등학교', '고등학생']


In [24]:
print("훈련 데이터 점수 : ", result.score(x_train, y_train))
print("테스트 데이터 예측 값 : ", result.predict(x_test))
print("테스트 데이터 점수 : ", result.score(x_test, y_test))

훈련 데이터 점수 :  0.8861533333333333
테스트 데이터 예측 값 :  [1 0 0 ... 1 0 1]
테스트 데이터 점수 :  0.85176


# LSTM을 이용한 분석

In [2]:
text_data, y_data = df_data['document'].as_matrix(), df_data['label'].as_matrix()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [3]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import numpy as np
import pickle
import os.path
import keras.backend.tensorflow_backend as K

# 토큰 파서
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
twitter_tag = Twitter()

In [5]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer)

In [6]:
!ls X*

X_data.pickle	 X_test_2.pickle  X_train_1.pickle
X_test_1.pickle  X_test.pickle	  X_train_2.pickle


In [7]:
# Tfidf 생성과 저장 - 단 파일이 없을 때만 !!
if not os.path.isfile("X_data.pickle"): 
    print('file does not exists')
    X_data = cv.fit_transform(text_data)
    pickle.dump(X_train, open("X_data.pickle", "wb"))

In [8]:
# 저장된 tfidf vector 데이터 읽기
with open('X_data.pickle', 'rb') as f:
    X_data = pickle.load(f)

In [9]:
!ls -al X*

-rw-rw-r-- 1 hjkim hjkim 24433014 Oct 28 18:39 X_data.pickle
-rw-rw-r-- 1 hjkim hjkim  7782344 Oct 28 19:03 X_test_1.pickle
-rw-rw-r-- 1 hjkim hjkim  5610600 Oct 28 18:56 X_test_2.pickle
-rw-rw-r-- 1 hjkim hjkim  8170628 Oct 28 18:40 X_test.pickle
-rw-rw-r-- 1 hjkim hjkim 23745856 Oct 28 19:02 X_train_1.pickle
-rw-rw-r-- 1 hjkim hjkim 14084384 Oct 28 18:55 X_train_2.pickle


In [10]:
# one-hot encoding
Y_data = np_utils.to_categorical(y_data, 2)

In [11]:
X_train = X_data[:100000]
X_test = X_data[100000:]

Y_train = Y_data[:100000]
Y_test = Y_data[100000:]

In [12]:
max_words = 61070 
nb_classes = 2
batch_size = 1024
nb_epoch = 5

In [13]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
Y_train.shape[1]

(100000, 104160)
(50000, 104160)
(100000, 2)


2

In [14]:
# LSTM 학습을 위한 데이터 재배열 (Time step)
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape)
print(X_test_rnn.shape)

(100000, 1, 104160)
(50000, 1, 104160)


In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

def build_LSTM_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(Y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
import keras.backend.tensorflow_backend as K
with K.tf.device('/GPU:1'):
    model_lstm = KerasClassifier(
    build_fn=build_LSTM_model, 
    epochs=nb_epoch, 
    batch_size=batch_size)
    
    model_lstm.fit(X_train_rnn, Y_train)

Epoch 1/5
100000/100000 [==============================] - 252s 3ms/step - loss: 0.5463 - acc: 0.7826
Epoch 2/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.2854 - acc: 0.8830
Epoch 3/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.2076 - acc: 0.9218
Epoch 4/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.1604 - acc: 0.9420
Epoch 5/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.1293 - acc: 0.9543


In [17]:
y = model_lstm.predict(X_train_rnn)
y_train = y_data[:100000]
ac_score = metrics.accuracy_score(y_train, y)
print("훈련 셋 정답률 =", ac_score)

훈련 셋 정답률 = 0.96941


In [18]:
# predict 함수는 예측 결과를 0 or 1로 출력하므로
# 학습과정에서 사용한 Y_train, Y_test 변수로 정확도 측정이 안됨
# Y_train, Y_test는 [0, 1], [1, 0]의 형태로 해당하는 감정 컬럼(class)은 1, 다른 컬럼은 0으로 표시됨
# 초기 y_data에 저장된 값을 그대로 활용하여 정확도를 측정

print("y : ", y)
print("Y_train[0] : ", Y_train)
print("y_train[0] : ", y_data)

y :  [0 1 0 ... 0 1 1]
Y_train[0] :  [[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
y_train[0] :  [0 1 0 ... 0 1 0]


In [19]:
y = model_lstm.predict(X_test_rnn)
y_test = y_data[100000:]
ac_score = metrics.accuracy_score(y_test, y)
print("테스트 셋 정답률 =", ac_score)

테스트 셋 정답률 = 0.83784
